<a href="https://colab.research.google.com/github/nanhmtnn/CS4200-Chess-Game/blob/main/Chess_Game_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies (if missing)
!pip install python-chess tensorflow scikit-learn ipywidgets --quiet

# Enable widgets in Colab
from google.colab import output
output.enable_custom_widget_manager()

# Create folder structure
!mkdir -p chess_ml_app/{data,models,ml_training,src}

# Verify folders
!tree chess_ml_app


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.3 MB/s eta 0:00:00
/bin/bash: line 1: tree: command not found


In [4]:
# =======================
# TRAIN ML CHESS EVALUATOR (Robust Version)
# =======================

import numpy as np
import pandas as pd
import chess
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# 1️⃣ Load Dataset
df = pd.read_csv("/content/chess_ml_app/data/chess_evaluations.csv")
print(f"Loaded {len(df)} positions.")

# 2️⃣ FEN Encoding
piece_to_index = {
    'P':0, 'N':1, 'B':2, 'R':3, 'Q':4, 'K':5,
    'p':6, 'n':7, 'b':8, 'r':9, 'q':10, 'k':11
}

def fen_to_vector(fen: str) -> np.ndarray:
    board = chess.Board(fen)
    arr = np.zeros((8,8,12), dtype=np.int8)
    for sq, piece in board.piece_map().items():
        row = 7 - chess.square_rank(sq)
        col = chess.square_file(sq)
        arr[row,col,piece_to_index[piece.symbol()]] = 1
    return arr.flatten()

print("Encoding FEN positions...")

valid_features = []
valid_scores = []
skipped = 0

for fen, score in zip(df["FEN"], df["Evaluation"]):
    try:
        # --- Skip invalid evaluation strings ---
        if isinstance(score, str):
            # remove possible spaces
            s = score.strip()
            # if it starts with '#' or not numeric, skip
            if s.startswith("#") or not any(ch.isdigit() for ch in s):
                skipped += 1
                continue
            score_val = float(s.replace(",", "").replace("+", ""))
        else:
            score_val = float(score)

        # --- Convert FEN safely ---
        vec = fen_to_vector(fen)
        valid_features.append(vec)
        valid_scores.append(score_val)

    except Exception:
        skipped += 1
        continue  # skip bad FENs or conversions

X = np.array(valid_features)
y = np.array(valid_scores, dtype=float)

print(f"✅ Encoded {len(X)} valid positions (skipped {skipped}).")

# 3️⃣ Normalize Targets
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y.reshape(-1,1)).flatten()

# 4️⃣ Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 5️⃣ Build Model
model = keras.Sequential([
    layers.Input(shape=(768,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 6️⃣ Train
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# 7️⃣ Save
model.save("/content/chess_ml_app/models/chess_eval_model.h5")
np.save("/content/chess_ml_app/models/eval_scaler_mean.npy", scaler_y.mean_)
np.save("/content/chess_ml_app/models/eval_scaler_std.npy", scaler_y.scale_)

print("✅ Model and scalers saved under /content/chess_ml_app/models/")


Loaded 338099 positions.
Encoding FEN positions...
✅ Encoded 335584 valid positions (skipped 2515).
Epoch 1/10
4720/4720 ━━━━━━━━━━━━━━━━━━━━ 69s 14ms/step - loss: 0.7751 - mae: 0.3329 - val_loss: 0.4863 - val_mae: 0.2827
Epoch 2/10
4720/4720 ━━━━━━━━━━━━━━━━━━━━ 64s 14ms/step - loss: 0.3265 - mae: 0.2523 - val_loss: 0.4100 - val_mae: 0.2583
Epoch 3/10
4720/4720 ━━━━━━━━━━━━━━━━━━━━ 65s 14ms/step - loss: 0.2234 - mae: 0.2183 - val_loss: 0.3883 - val_mae: 0.2386
Epoch 4/10
4720/4720 ━━━━━━━━━━━━━━━━━━━━ 63s 13ms/step - loss: 0.1735 - mae: 0.1976 - val_loss: 0.3726 - val_mae: 0.2374
Epoch 5/10
4720/4720 ━━━━━━━━━━━━━━━━━━━━ 66s 14ms/step - loss: 0.1450 - mae: 0.1847 - val_loss: 0.3460 - val_mae: 0.2185
Epoch 6/10
4720/4720 ━━━━━━━━━━━━━━━━━━━━ 85s 14ms/step - loss: 0.1210 - mae: 0.1710 - val_loss: 0.3533 - val_mae: 0.2199
Epoch 7/10
4720/4720 ━━━━━━━━━━━━━━━━━━━━ 65s 14ms/step - loss: 0.1069 - mae: 0.1620 - val_loss: 0.3597 - val_mae: 0.2171
Epoch 8/10
4720/4720 ━━━━━━━━━━━━━━━━━━━━ 64s 

✅ Model and scalers saved under /content/chess_ml_app/models/


In [6]:
# =======================
# STEP 4 — ML EVALUATOR (Fixed for Colab TF 2.17+)
# =======================

import numpy as np
import chess
from tensorflow.keras.models import load_model

# Load model safely (avoid 'mse' serialization bug)
ml_model = load_model("/content/chess_ml_app/models/chess_eval_model.h5", compile=False)

# Load normalization stats
eval_mean = np.load("/content/chess_ml_app/models/eval_scaler_mean.npy", allow_pickle=True)
eval_std = np.load("/content/chess_ml_app/models/eval_scaler_std.npy", allow_pickle=True)

piece_to_index = {
    'P':0,'N':1,'B':2,'R':3,'Q':4,'K':5,
    'p':6,'n':7,'b':8,'r':9,'q':10,'k':11
}

def board_to_vector(board: chess.Board) -> np.ndarray:
    arr = np.zeros((8,8,12), dtype=np.int8)
    for sq,piece in board.piece_map().items():
        row = 7 - chess.square_rank(sq)
        col = chess.square_file(sq)
        arr[row,col,piece_to_index[piece.symbol()]] = 1
    return arr.flatten().reshape(1,-1)

def evaluate_board_raw(board: chess.Board) -> float:
    vec = board_to_vector(board)
    pred = ml_model.predict(vec, verbose=0)[0][0]
    score = float(pred * eval_std + eval_mean)
    return score


In [7]:
# =======================
# CHESS ENGINE (AI MOVE)
# =======================

import math
from typing import Optional
from chess import Board, Move, WHITE
from IPython.display import display

def eval_for_color(board: Board, ai_color: bool) -> float:
    raw = evaluate_board_raw(board)
    return raw if ai_color == WHITE else -raw

def best_move(board: Board, depth: int, ai_color: bool) -> Optional[Move]:
    """Select the move with highest predicted evaluation."""
    best_mv, best_score = None, -math.inf
    for mv in board.legal_moves:
        board.push(mv)
        score = eval_for_color(board, ai_color)
        board.pop()
        if score > best_score:
            best_score, best_mv = score, mv
    return best_mv


In [16]:
from __main__ import best_move   # since everything is in one Colab notebook
import math
from dataclasses import dataclass
from typing import Optional, Tuple, List, Dict

import chess
import ipywidgets as W
from IPython.display import display, HTML
# -------------------------------
#           UI (ipywidgets)
# -------------------------------

# Piece glyphs
GLYPH = {
    'P':'♙','N':'♘','B':'♗','R':'♖','Q':'♕','K':'♔',
    'p':'♟','n':'♞','b':'♝','r':'♜','q':'♛','k':'♚',
}
# LIGHT = '#F0D9B5'
# DARK  = '#B58863'
LIGHT = '#ffca7b'
DARK  = '#ff7251'
SEL   = '#f6f67a'
TARGET= '#b9e6a1'
CAPT  = '#f5a3a3'

# Optional: bump button font-size
display(HTML("<style>.widget-button{font-size:22px !important;}</style>"))
# Optional: bump button font-size + enforce black text
display(HTML("""
<style>
.widget-button {
    color: black !important;
    font-size: 24px !important;
    font-weight: bold !important;
}
</style>
"""))


@dataclass
class GameState:
    board: chess.Board
    ai_color: Optional[chess.Color]  # None for 2-player mode
    depth: int
    orientation_white: bool  # True = white-bottom view

    def status_text(self) -> str:
        if self.board.is_game_over():
            outcome = self.board.outcome()
            if outcome is None or outcome.winner is None:
                return "Game over · Draw"
            return "Game over · " + ("White wins" if outcome.winner == chess.WHITE else "Black wins")
        who = "White" if self.board.turn == chess.WHITE else "Black"
        check = " · Check!" if self.board.is_check() else ""
        return f"Turn: {who}{check}"

class ChessApp:
    def __init__(self):
        # Controls
        self.mode = W.ToggleButtons(options=[('Vs AI','ai'),('Two Players','2p')], value='ai', description='Mode:')
        self.color_sel = W.ToggleButtons(options=[('White','white'),('Black','black')], value='white', description='You:')
        self.depth = W.IntSlider(value=3, min=2, max=5, step=1, description='AI depth:')
        self.start_btn = W.Button(description='Start / Reset', button_style='primary')
        self.undo_btn = W.Button(description='Undo')
        self.flip_btn = W.Button(description='Flip')
        self.status = W.HTML("<b>Click Start / Reset to begin.</b>")
        self.log = W.HTML("")
        self.log_box = W.VBox([W.HTML("<b>Moves</b>"), W.Box([self.log], layout=W.Layout(max_height='220px', overflow='auto'))])

        # Promotion chooser (hidden until needed)
        self.promo_box = W.HBox([])
        self._promo_pending = None  # (from_sq, to_sq)

        # Board grid
        self.grid_box = W.GridBox(children=[], layout=W.Layout(grid_template_columns='repeat(8, 46px)', grid_gap='0px'))
        self.sq_buttons: Dict[int, W.Button] = {}  # square -> button
        self.selected_sq: Optional[int] = None
        self.legal_from_selected: List[chess.Move] = []

        # State
        self.state: Optional[GameState] = None

        # Wire controls
        self.start_btn.on_click(self.on_start)
        self.undo_btn.on_click(self.on_undo)
        self.flip_btn.on_click(self.on_flip)
        self.mode.observe(self.on_mode_change, 'value')

        # Build panel
        top = W.HBox([self.mode, self.color_sel, self.depth, self.start_btn, self.undo_btn, self.flip_btn])
        self.ui = W.VBox([top, self.status, self.promo_box, W.HBox([self.grid_box, self.log_box])])

        self._build_empty_grid()
        display(self.ui)

    # ---------- helpers ----------
    def _square_to_rc(self, sq: int) -> Tuple[int,int]:
        """Return (row, col) on the 8x8 grid for a given 0..63 square, given orientation."""
        file = chess.square_file(sq)
        rank = chess.square_rank(sq)
        if self.state and self.state.orientation_white:
            row = 7 - rank
            col = file
        else:
            row = rank
            col = 7 - file
        return row, col

    def _rc_to_square(self, row: int, col: int) -> int:
        """Return 0..63 square for grid (row, col)."""
        if self.state and self.state.orientation_white:
            rank = 7 - row
            file = col
        else:
            rank = row
            file = 7 - col
        return chess.square(file, rank)

    def _build_empty_grid(self):
        self.grid_box.children = ()
        self.sq_buttons.clear()
        children = []
        for row in range(8):
            for col in range(8):
                sq = self._rc_to_square(row, col)
                light = (row + col) % 2 == 0
                b = W.Button(description=' ', layout=W.Layout(width='46px', height='46px', padding='0'),
                             tooltip=chess.square_name(sq))
                b.style.button_color = LIGHT if light else DARK
                b.on_click(self._make_square_click(sq))
                self.sq_buttons[sq] = b
                children.append(b)
        self.grid_box.children = tuple(children)

    def _render_board(self):
        board = self.state.board
        # Clear any highlights if selection disappeared
        if self.selected_sq is not None and (self.selected_sq not in [m.from_square for m in board.legal_moves]):
            self.selected_sq = None
            self.legal_from_selected = []
        # Set base colors & glyphs
        for sq, btn in self.sq_buttons.items():
            row, col = self._square_to_rc(sq)
            light = (row + col) % 2 == 0
            btn.style.button_color = LIGHT if light else DARK
            piece = board.piece_at(sq)
            btn.description = GLYPH.get(piece.symbol(), ' ') if piece else ' '
        # Highlight selection and legal targets
        if self.selected_sq is not None:
            self.sq_buttons[self.selected_sq].style.button_color = SEL
            targets = [m.to_square for m in self.legal_from_selected]
            for m in self.legal_from_selected:
                tgt = m.to_square
                btn = self.sq_buttons[tgt]
                btn.style.button_color = CAPT if self.state.board.is_capture(m) else TARGET

        # Update status
        self.status.value = f"<b>{self.state.status_text()}</b>"

    def _append_log(self, text: str):
        self.log.value += f"{text}<br>"
        # cap log length a bit
        if self.log.value.count("<br>") > 200:
            self.log.value = "<i>(trimmed)</i><br>" + "<br>".join(self.log.value.split("<br>")[-200:])

    def _make_square_click(self, sq: int):
        def handler(_):
            if self.state is None or self.state.board.is_game_over():
                return
            # If promotion dialog visible, ignore board clicks
            if self._promo_pending is not None:
                return

            board = self.state.board

            # Whose turn is allowed to move?
            if self.state.ai_color is not None:
                human_turn = (board.turn != self.state.ai_color)
                if not human_turn:
                    return  # wait for AI
            # In 2-player mode both can move

            piece = board.piece_at(sq)
            if self.selected_sq is None:
                # First click: must be a piece of side to move
                if piece is None or piece.color != board.turn:
                    return
                self.selected_sq = sq
                self.legal_from_selected = [m for m in board.legal_moves if m.from_square == sq]
                self._render_board()
                return
            else:
                # Second click: attempt move
                if sq == self.selected_sq:
                    self.selected_sq = None
                    self.legal_from_selected = []
                    self._render_board()
                    return

                # Allow reselecting another own piece
                if piece is not None and piece.color == board.turn:
                    self.selected_sq = sq
                    self.legal_from_selected = [m for m in board.legal_moves if m.from_square == sq]
                    self._render_board()
                    return

                # Try to move selected -> sq (with possible promotion)
                legal_targets = [m for m in self.legal_from_selected if m.to_square == sq]
                if not legal_targets:
                    return

                # Promotion handling: if any candidate has a promotion, ask
                promos = [m for m in legal_targets if m.promotion]
                if promos:
                    self._prompt_promotion(self.selected_sq, sq, promos)
                    return

                # Otherwise, there should be exactly one legal move
                mv = legal_targets[0]
                self._play_human_move(mv)
        return handler

    def _prompt_promotion(self, from_sq: int, to_sq: int, promo_moves: List[chess.Move]):
        # Show small chooser
        self._promo_pending = (from_sq, to_sq)
        opts = [('Queen','q'),('Rook','r'),('Bishop','b'),('Knight','n')]
        picker = W.ToggleButtons(options=opts, value='q', description='Promote:')
        ok_btn = W.Button(description='OK', button_style='success')
        cancel_btn = W.Button(description='Cancel')
        msg = W.HTML("")

        def on_ok(_):
            letter = picker.value
            promo_map = {'q':chess.QUEEN,'r':chess.ROOK,'b':chess.BISHOP,'n':chess.KNIGHT}
            promo_piece = promo_map[letter]
            # find matching legal move
            for m in promo_moves:
                if m.from_square == from_sq and m.to_square == to_sq and m.promotion == promo_piece:
                    self._clear_promo_ui()
                    self._play_human_move(m)
                    return
            msg.value = "<span style='color:#b91c1c'>Not a legal promotion.</span>"

        def on_cancel(_):
            self._clear_promo_ui()
            # keep selection so user can try again

        ok_btn.on_click(on_ok)
        cancel_btn.on_click(on_cancel)
        self.promo_box.children = [picker, ok_btn, cancel_btn, msg]

    def _clear_promo_ui(self):
        self.promo_box.children = []
        self._promo_pending = None

    # ---------- actions ----------
    def on_start(self, _):
        human_color = self.color_sel.value
        ai = (None if self.mode.value == '2p'
              else (chess.BLACK if human_color == 'white' else chess.WHITE))
        self.state = GameState(board=chess.Board(), ai_color=ai, depth=int(self.depth.value),
                               orientation_white=True if human_color=='white' else False)
        self.selected_sq = None
        self.legal_from_selected = []
        self._build_empty_grid()
        self.log.value = ""
        self._clear_promo_ui()

        # If AI plays first (AI = white)
        if self.state.ai_color == chess.WHITE and self.state.board.turn == chess.WHITE:
            mv = best_move(self.state.board, self.state.depth, self.state.ai_color)
            if mv:
                san = self.state.board.san(mv)
                self.state.board.push(mv)
                self._append_log(f"AI: {san}")
        self._render_board()

    def on_undo(self, _):
        if not self.state: return
        b = self.state.board
        undone = 0
        if len(b.move_stack): b.pop(); undone += 1
        if self.state.ai_color is not None and len(b.move_stack):
            # try undo a full pair vs AI
            b.pop(); undone += 1
        self.selected_sq = None
        self.legal_from_selected = []
        self._append_log(f"<i>Undid {undone} half-move(s).</i>")
        self._render_board()

    def on_flip(self, _):
        if not self.state: return
        self.state.orientation_white = not self.state.orientation_white
        self._build_empty_grid()
        self._render_board()

    def on_mode_change(self, change):
        # Enable/disable controls based on mode
        is_ai = change['new'] == 'ai'
        self.color_sel.disabled = not is_ai
        self.depth.disabled = not is_ai

    def _play_human_move(self, mv: chess.Move):
        b = self.state.board
        san = b.san(mv)
        b.push(mv)
        self._append_log(f"You: {san}")
        self.selected_sq = None
        self.legal_from_selected = []
        self._render_board()
        # If game not over and vs AI, let AI respond
        if not b.is_game_over() and self.state.ai_color is not None and b.turn == self.state.ai_color:
            reply = best_move(b, self.state.depth, self.state.ai_color)
            if reply:
                san2 = b.san(reply)
                b.push(reply)
                self._append_log(f"AI: {san2}")
                self._render_board()

In [17]:
# =======================
# RUN THE CHESS APP
# =======================
ChessApp()


/tmp/ipython-input-4163183855.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score = float(pred * eval_std + eval_mean)
/tmp/ipython-input-4163183855.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score = float(pred * eval_std + eval_mean)
/tmp/ipython-input-4163183855.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score = float(pred * eval_std + eval_mean)
/tmp/ipython-input-4163183855.py:32: DeprecationWarning: Conversion of an array with ndim >